#**Deep Learning homework**

In [24]:
import os
import shutil
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchsummary import summary

In [25]:
# Reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [26]:
# Remove old zip if exists
if os.path.exists("data.zip"):
    os.remove("data.zip")

# Download new data.zip
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip -O data.zip

# Remove existing data/ folder
if os.path.exists("data"):
    shutil.rmtree("data")

# Unzip the new dataset
!unzip -q data.zip

--2025-11-21 11:25:58--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-21T12%3A20%3A05Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-21T11%3A19%3A06Z&ske=2025-11-21T12%3A20%3A05Z&sks=b&skv=2018-11-09&sig=xnp2e3ClLgQtkfyiNNn8vH2EmiW0Rt%2BJOj0MrAMjNAc%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2MzcyNjE1OCwibmJmIjoxNzYzNzI0MzU4LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5i

In [27]:
!find data -type d | sed 's/[^-][^\/]*\//    /g;s/\//|-- /'

data
    test
        straight
        curly
    train
        straight
        curly


In [28]:
data_dir = "data"
train_dir = os.path.join(data_dir, "train")
test_dir  = os.path.join(data_dir, "test")

In [29]:
# Resize to 200x200, convert to tensor
train_transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
])

In [30]:
# Datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_dataset  = datasets.ImageFolder(root=test_dir, transform=test_transform)

In [31]:
# DataLoaders
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class_names = train_dataset.classes
print("Classes:", class_names)

Classes: ['curly', 'straight']


In [32]:
model = nn.Sequential(
    # Conv2d: in_channels=3, out=32, kernel=3x3
    nn.Conv2d(
        in_channels=3,
        out_channels=32,
        kernel_size=3
    ),
    nn.ReLU(),

    # MaxPool2d: 2x2
    nn.MaxPool2d(kernel_size=2, stride=2),

    # Flatten
    nn.Flatten(),

    # Fully-connected 64 units + ReLU
    nn.Linear(32 * 99 * 99, 64),
    nn.ReLU(),

    # Output layer: 1 neuron
    nn.Linear(64, 1),
)

print(model)

Sequential(
  (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Flatten(start_dim=1, end_dim=-1)
  (4): Linear(in_features=313632, out_features=64, bias=True)
  (5): ReLU()
  (6): Linear(in_features=64, out_features=1, bias=True)
)


In [33]:
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.002,
    momentum=0.8
)

###**<font color='red'>Question 1</font>**
Which loss function you will use?
- nn.MSELoss()
- <font color='green'>nn.BCEWithLogitsLoss()</font> ✅
- nn.CrossEntropyLoss()
- nn.CosineEmbeddingLoss()

(Multiple answered can be correct, so pick any)

In [34]:
criterion = nn.BCEWithLogitsLoss()

###**<font color='red'>Question 2</font>**
What's the total number of parameters of the model?

- 896
- 11214912
- 15896912
- <font color='green'>20073473</font> ✅

In [35]:
summary(model, input_size=(3, 200, 200))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
              ReLU-2         [-1, 32, 198, 198]               0
         MaxPool2d-3           [-1, 32, 99, 99]               0
           Flatten-4               [-1, 313632]               0
            Linear-5                   [-1, 64]      20,072,512
              ReLU-6                   [-1, 64]               0
            Linear-7                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 23.93
Params size (MB): 76.57
Estimated Total Size (MB): 100.96
----------------------------------------------------------------


###**<font color='red'>Question 3</font>**
What is the median of training accuracy for all the epochs for this model?

- 0.05
- 0.12
- 0.40
- <font color='green'>0.84</font> ✅

In [36]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [37]:
# Datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
test_dataset  = datasets.ImageFolder(root=test_dir, transform=train_transforms)

In [38]:
# DataLoaders
batch_size = 20

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [39]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'test_acc': [], 'test_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history['test_loss'].append(val_epoch_loss)
    history['test_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6665, Acc: 0.6112, Val Loss: 0.6511, Val Acc: 0.6617
Epoch 2/10, Loss: 0.5702, Acc: 0.6787, Val Loss: 0.6332, Val Acc: 0.6318
Epoch 3/10, Loss: 0.5207, Acc: 0.7350, Val Loss: 0.6143, Val Acc: 0.6766
Epoch 4/10, Loss: 0.4773, Acc: 0.7600, Val Loss: 0.6049, Val Acc: 0.6617
Epoch 5/10, Loss: 0.4606, Acc: 0.7550, Val Loss: 0.7307, Val Acc: 0.5672
Epoch 6/10, Loss: 0.3954, Acc: 0.8275, Val Loss: 0.6412, Val Acc: 0.6866
Epoch 7/10, Loss: 0.2844, Acc: 0.8838, Val Loss: 0.8307, Val Acc: 0.6816
Epoch 8/10, Loss: 0.2885, Acc: 0.8788, Val Loss: 0.7052, Val Acc: 0.7114
Epoch 9/10, Loss: 0.1882, Acc: 0.9313, Val Loss: 0.9275, Val Acc: 0.6866
Epoch 10/10, Loss: 0.2585, Acc: 0.8912, Val Loss: 0.8158, Val Acc: 0.6915


In [40]:
arr = np.array(history["acc"])
median = np.median(arr).round(2)
print(median)

0.79


###**<font color='red'>Question 4</font>**
What is the standard deviation of training loss for all the epochs for this model?

- 0.007
- 0.078
- <font color='green'>0.171</font> ✅
- 1.710

In [41]:
arr = np.array(history["loss"])
std = np.std(arr, ddof=1).round(3)
print(std)

0.154


###**<font color='red'>Question 5</font>**

What is the mean of test loss for all the epochs for the model trained with augmentations?

- 0.008
- 0.08
- <font color='green'>0.88</font> ✅
- 8.88

In [42]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip()
])

In [43]:
# Datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
test_dataset  = datasets.ImageFolder(root=test_dir, transform=train_transforms)

In [44]:
# DataLoaders
batch_size = 20

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [45]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'test_acc': [], 'test_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history['test_loss'].append(val_epoch_loss)
    history['test_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6409, Acc: 0.6587, Val Loss: 0.5929, Val Acc: 0.7114
Epoch 2/10, Loss: 0.5727, Acc: 0.7100, Val Loss: 0.6892, Val Acc: 0.6219
Epoch 3/10, Loss: 0.5820, Acc: 0.7025, Val Loss: 0.5818, Val Acc: 0.7164
Epoch 4/10, Loss: 0.5502, Acc: 0.6987, Val Loss: 0.5941, Val Acc: 0.6667
Epoch 5/10, Loss: 0.5238, Acc: 0.7412, Val Loss: 0.5915, Val Acc: 0.6766
Epoch 6/10, Loss: 0.5263, Acc: 0.7412, Val Loss: 0.5690, Val Acc: 0.7065
Epoch 7/10, Loss: 0.5173, Acc: 0.7500, Val Loss: 0.5935, Val Acc: 0.6667
Epoch 8/10, Loss: 0.5008, Acc: 0.7438, Val Loss: 0.5624, Val Acc: 0.6567
Epoch 9/10, Loss: 0.5066, Acc: 0.7288, Val Loss: 0.5794, Val Acc: 0.7313
Epoch 10/10, Loss: 0.5111, Acc: 0.7400, Val Loss: 0.5683, Val Acc: 0.7363


In [46]:
arr = np.array(history["test_loss"])
mean = np.mean(arr).round(3)
print(mean)

0.592


###**<font color='red'>Question 6</font>**
What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

- 0.08
- 0.28
- <font color='green'>0.68</font> ✅
- 0.98




In [49]:
arr = np.array(history["acc"][5:])
mean = np.mean(arr).round(3)
print(mean)

0.741
